In [1]:
import math
import os
import os.path
import collections
import numpy as np
import pandas as pd
from scipy import interpolate

In [2]:
devices_dir = '/home/pravindran/mycode/tracolo/data/devices/'

devices = {'FLMS00114': {'coeffs': np.loadtxt(os.path.join(devices_dir, 'FLMS00114_coeffs_raw.txt'), delimiter=';'),
                         'upwell_fov_angle': 25, 
                         'dnwell_fov_angle': 25,
                         'upwell_fiber_radius': 200, 
                         'dnwell_fiber_radius': 300,
                         'upwell_factor': 1.0,
                         'dnwell_factor': 2.0*math.pi}, 
           'NQ51B0141': {'coeffs': np.loadtxt(os.path.join(devices_dir, 'NQ51B0141_coeffs_raw.txt'), delimiter=';'),
                         'upwell_fov_angle': 25, 
                         'dnwell_fov_angle': 25,
                         'upwell_fiber_radius': 200, 
                         'dnwell_fiber_radius': 300,
                         'upwell_factor': 1.0,
                         'dnwell_factor': 2.0*math.pi}}

In [3]:
# Path to data directories
# sds
tractor_dir = '/home/pravindran/mycode/tracolo/data/sds/Tractor/'
piccolo_dir = '/home/pravindran/mycode/tracolo/data/sds/Piccolo/'
inter_dir = '/home/pravindran/mycode/tracolo/data/sds/interpolated/'

In [4]:
def is_numeric(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [5]:
def to_seconds(t):
    '''
    Convert HH:MM:SS to seconds since start of day.
    '''
    pcs = str(t).split(':')
    h, m, s = float(pcs[0]), float(pcs[1]), float(pcs[2])
    return (h*60.0 + m)*60.0 + s

In [6]:
# NOTE: emergence-->sds::plot-->Plot
def get_tractor_dataframe(filepath, print_details=False):
    '''
    Reads tractor csv file.
    Returns exp_id, date and pd.DataFrame.
    Add date_stamp and time_stamp columns to pd.DataFrame.
    '''
    # Get the experiment id
    tokens1 = os.path.split(filepath)
    tokens2 = os.path.split(tokens1[0])
    exp_id = tokens2[1]
    
    # Read the csv into pd.DataFrame
    df = pd.read_csv(filepath)
    # Convert date time to number of seconds since start of day.
    df['GPS_dt'] = pd.to_datetime(df['GPS_Date'] + " " + df['GPS_Time'])
    df['date_stamp'] = df['GPS_dt'].dt.date
    df['time_stamp'] = df['GPS_dt'].dt.time
    date = str(df['date_stamp'].unique()[0])
    df['secs_stamp'] = df['time_stamp'].apply(to_seconds)
    # Sort by Point_ID to handle the unsorted time
    df = df.sort_values('Point_ID', axis=0)
    # Compute cluster ids for plot number runs.
    # The tractor can revisit same plot more than once.
    diffs = np.zeros(df.shape[0], dtype=np.int)
    diffs[1:] = np.abs(np.diff(df['Plot'].values))
    diffs[0] = 1
    diffs[diffs > 0] = 1
    df['cluster_id'] = np.cumsum(diffs)
    # Reset index for data frame
    df.index = np.arange(df.shape[0])
    
    # Print some details
    if print_details:
        print("{}:{}".format(exp_id, date))
        print("\tdf.shape: {}".format(df.shape))
        print(df['GPS_dt'].head())
        print(df['date_stamp'].head())
        print(df['time_stamp'].head())
        print("--------------------------------")
    
    
    return exp_id, date, df[['Point_ID', 'cluster_id', 'Plot', 
                             'secs_stamp', 'Latitude', 'Longitude', 
                             'date_stamp', 'time_stamp']]

In [7]:
def get_calibrated_spectrum_row(group, wave_cols, nonwave_cols, devices):
    '''
    Computes the calibrated spectrum. 
    Creates a new row from 4 rows.
    '''
    g = group[1]
    d = devices[group[1]['device'].values[0]]
    
    # Normalize up
    upwell_meas_row = g[(g['direction'] == 'Upwelling') & (g['dark'] == False)]
    upwell_dark_row = g[(g['direction'] == 'Upwelling') & (g['dark'] == True)]
    #### up parameters
    upwell_area = math.pi*d['upwell_fiber_radius']*d['upwell_fiber_radius']
    upwell_fov = d['upwell_fov_angle']*math.pi/180.0
    upwell_factor = d['upwell_factor']
    upwell_time = upwell_meas_row['integration_time'].values[0]
    #### up dark corrected and normalized
    upwell_dvsr = upwell_time*upwell_area*upwell_fov*upwell_factor
    upwell_meas = np.array(upwell_meas_row[wave_cols].values, dtype=np.double)
    upwell_dark = np.array(upwell_dark_row[wave_cols].values, dtype=np.double)
    upwell_norm = (upwell_meas.flatten() - upwell_dark.flatten())/upwell_dvsr
    
    # Normalize dn
    dnwell_meas_row = g[(g['direction'] == 'Downwelling') & (g['dark'] == False)]
    dnwell_dark_row = g[(g['direction'] == 'Downwelling') & (g['dark'] == True)]
    #### dn parameters
    dnwell_area = math.pi*d['dnwell_fiber_radius']*d['dnwell_fiber_radius']
    dnwell_fov = d['dnwell_fov_angle']*math.pi/180.0
    dnwell_factor = d['dnwell_factor']
    dnwell_time = dnwell_meas_row['integration_time'].values[0]
    #### dn dark corrected and normalized
    dnwell_dvsr = dnwell_time*dnwell_area*dnwell_fov*dnwell_factor
    dnwell_meas = np.array(dnwell_meas_row[wave_cols].values, dtype=np.double)
    dnwell_dark = np.array(dnwell_dark_row[wave_cols].values, dtype=np.double)
    dnwell_norm = (dnwell_meas.flatten() - dnwell_dark.flatten())/dnwell_dvsr
    
    # NOTE: Modifications for SDS.
    # Reflectance.
    not_nan_idxs = np.logical_not(np.isnan(upwell_norm))
    not_nan_refls = list(upwell_norm[not_nan_idxs]*d['coeffs']/dnwell_norm[not_nan_idxs])
    refls = np.zeros(upwell_norm.shape, dtype=upwell_norm.dtype)
    refls.fill(np.nan)
    refls[not_nan_idxs] = not_nan_refls
    refls = list(refls)
    
    # Retain columns
    others = list(upwell_meas_row[nonwave_cols].values[0])
    
    # Something weird is happening with the name of columns in in one of the csv files.
    # So enforce that all wavelengths are float type
    fwave_cols = [str(float(w)) for w in wave_cols]
    
    # set and return
    return pd.DataFrame([others + refls], columns=(nonwave_cols + fwave_cols))

In [8]:
def get_piccolo_dataframe(filepath, devices, print_details=False):
    '''
    Reads tractor csv file.
    Returns exp_id, file id and pd.DataFrame.
    Add date_stamp and time_stamp columns to pd.DataFrame.
    '''
    # Get the experiment id
    tokens1 = os.path.split(filepath)
    tokens2 = os.path.split(tokens1[0])
    exp_id = tokens2[1]
    
    # Read the dataframe. Add GPS_Date and GPS_Time columns to it.
    df = pd.read_csv(filepath, index_col=0)
    df['GPS_dt'] = pd.to_datetime(df['datetime'])
    df['date_stamp'] = df['GPS_dt'].dt.date
    df['time_stamp'] = df['GPS_dt'].dt.time
    df['secs_stamp'] = df['time_stamp'].apply(to_seconds)
    df['secs_stamp'] -= 3600.0
    date = str(df['date_stamp'].unique()[0])
    

    # Separate the columns into those that are wavelengths 
    # and those that are not wavelengths
    nonwave_cols = []
    wave_cols = []
    for c in df.columns:
        if is_numeric(str(c)):
            wave_cols.append(str(c))
        else:
            nonwave_cols.append(str(c))
            
    # There will be 4 or 8 rows per filename.
    # If 4, the device names should be the same.
    # If 8, two devices are present with four rows per device.
    grouped = df.groupby(['filename', 'device'], as_index=False)
    rdfs = []
    for g in grouped:
        row = get_calibrated_spectrum_row(group=g,
                                          wave_cols=wave_cols,
                                          nonwave_cols=nonwave_cols,
                                          devices=devices)
        rdfs.append(row)
    
    rdf = pd.concat(rdfs)
    rdf.reset_index()
#     print(rdf.columns[10:20])
    
    # Print some details
    if print_details:
        print("{}:{}".format(exp_id, date))
        print("\tdf.shape: {}".format(df.shape))
        print(df['GPS_dt'].head())
        print(df['date_stamp'].head())
        print(df['time_stamp'].head())
        print("--------------------------------")
        
    return exp_id, date, rdf

In [9]:
# Load Tractor files
tractor_subdirs = os.listdir(tractor_dir)
tractor_subdirs.sort()
tractor_subdirs = [os.path.join(tractor_dir, d) for d in tractor_subdirs]
tractor_dfs = collections.OrderedDict()
for sd in tractor_subdirs:
    # Get the exp* sub-directories
    filenames = os.listdir(sd)
    filenames.sort()
    # The files in the exp* directory are loaded.
    # Saved in OrderedDict with key exp_id:file_id
    for fn in filenames:
        if fn[0] != '.':
            exp_id, date, df = get_tractor_dataframe(os.path.join(sd, fn), print_details=False)
            tractor_dfs[":".join([exp_id, date])] = df

print("The keys for tractor DataFrames are: ")
for i, k in enumerate(tractor_dfs):
    print("{}: {}: {}".format(i + 1, k, tractor_dfs[k].values.shape))

The keys for tractor DataFrames are: 
1: exp16045:2016-06-08: (4107, 8)
2: exp16045:2016-06-28: (5095, 8)
3: exp16045:2016-07-18: (4956, 8)
4: exp16045:2016-07-26: (5231, 8)
5: exp16045:2016-08-15: (5030, 8)
6: exp16045:2016-08-22: (4922, 8)
7: exp16045:2016-09-02: (4779, 8)
8: exp16045:2016-09-12: (2503, 8)


In [10]:
# Load Piccolo files
piccolo_subdirs = os.listdir(piccolo_dir)
piccolo_subdirs.sort()
piccolo_subdirs = [os.path.join(piccolo_dir, d) for d in piccolo_subdirs]
piccolo_piece_dfs = collections.OrderedDict()
for sd in piccolo_subdirs:
    # Get the exp* sub-directories
    filenames = os.listdir(sd)
    filenames.sort()
    # The files in the exp* directory are loaded.
    # Saved in OrderedDict with key exp_id:file_id
    for fn in filenames:
        print(fn)
        exp_id, date, df = get_piccolo_dataframe(os.path.join(sd, fn), 
                                                 devices=devices, 
                                                 print_details=False)
        key = "{}:{}".format(exp_id, date)
        if key in piccolo_piece_dfs:
            piccolo_piece_dfs[key].append(df)
        else:
            piccolo_piece_dfs[key] = [df]

# Merge piece dfs.
piccolo_dfs = collections.OrderedDict()
for k in piccolo_piece_dfs:
    piccolo_dfs[k] = pd.concat(piccolo_piece_dfs[k])
    nrows = piccolo_dfs[k].shape[0]
    print("nrows: {}".format(nrows))
    piccolo_dfs[k].index = np.arange(nrows)
    
# Print some stuff.    
print("The keys for piccolo DataFrames are: ")
for i, k in enumerate(piccolo_dfs):
    print("{}: {}: {}".format(i + 1, k, piccolo_dfs[k].values.shape))

C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160608_06.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160608_07.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160608_08.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160628_01.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160628_02.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160628_03.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160628_04.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160628_05.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160628_06.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160628_07.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160628_08.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160628_09.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160628_17.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160628_18.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160628_19.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160628_20.csv
C___Users_iherrmann.RUSSELL_Desktop_temp_S_20160628_21.c

In [11]:
# Do the interpolation
for k in piccolo_dfs:
    print(k)
    if k in tractor_dfs:
        pdf = piccolo_dfs[k]
        tdf = tractor_dfs[k]
        # print("{}: min:{},   max: {}".format(k, np.min(pdf['secs_stamp'].values), np.max(pdf['secs_stamp'].values)))
        # print("{}: min:{},   max: {}".format(k, np.min(tdf['secs_stamp'].values), np.max(tdf['secs_stamp'].values)))
        pdf_lat = np.zeros(pdf.shape[0], dtype=np.double)
        pdf_lon = np.zeros(pdf.shape[0], dtype=np.double)
        pdf_plt = np.zeros(pdf.shape[0], dtype=np.int)
        clusters = tractor_dfs[k].groupby('cluster_id')
        for cluster in clusters:
            if cluster[1]['secs_stamp'].values.size >= 4: 
                #             print(cluster[1]['secs_stamp'].values.shape, cluster[1]['Latitude'].values.shape)
                flat = interpolate.interp1d(cluster[1]['secs_stamp'].values, 
                                            cluster[1]['Latitude'].values, 
                                            bounds_error=False,
                                            fill_value=0.0)
                flon = interpolate.interp1d(cluster[1]['secs_stamp'].values, 
                                            cluster[1]['Longitude'].values, 
                                            bounds_error=False,
                                            fill_value=0.0)
                ilat = flat(pdf['secs_stamp'].values)
                ilon = flon(pdf['secs_stamp'].values)
                pdf_lat += ilat
                pdf_lon += ilon
                # emergence --> sds :: plot --> Plot
                pdf_plt[np.where(np.abs(ilat) > 0.0)] = cluster[1]['Plot'].unique()[0]
        piccolo_dfs[k]['latitude'] = pdf_lat
        piccolo_dfs[k]['longitude'] = pdf_lon
        piccolo_dfs[k]['plot'] = pdf_plt
        piccolo_dfs[k] = piccolo_dfs[k][piccolo_dfs[k]['plot'] > 0]

exp16045:2016-06-08
exp16045:2016-06-28
exp16045:2016-07-18
exp16045:2016-07-26
exp16045:2016-08-15
exp16045:2016-08-22
exp16045:2016-09-02
exp16045:2016-09-12


In [12]:
# Save the piccolo dataframes into files
import shutil
if os.path.exists(inter_dir):
    shutil.rmtree(inter_dir)
os.mkdir(inter_dir)
for (i, k) in enumerate(piccolo_dfs):
    cols1 = ['filename', 'device', 'secs_stamp', 'plot', 'latitude', 'longitude']
    cols2 = ['direction', 'dark', 'replicate', 'integration_time', 'datetime', 
             'GPS_dt', 'date_stamp', 'time_stamp']
    cols = piccolo_dfs[k].columns.values
    waves = []
    for c in cols:
        if c not in cols1 and c not in cols2:
            waves.append(c)
    piccolo_dfs[k].to_csv(os.path.join(inter_dir, k + '_piccolo.csv'),
                          columns=(cols1 + waves + cols2))